## Basic Combustion Analysis
Using Cantera to calculate combustion mechanics.
The one free variable in our design is chamber pressure, for now I will set that at 10 bars
We will be assuming stoichiometric toluene air combustion

In [27]:
import cantera as ct
import numpy as np

# ----- Parameters -----
tank_pressure = 10 * ct.one_atm  # 10 bar
A_throat = 4e-3 * 10e-3 

# ----- Use Cantera's built-in thermo with custom composition -----
# We'll compute enthalpy properly

gas = ct.Solution('gri30.yaml')

# Stoichiometric products (before equilibration)
# C7H8 + 9 O2 + 33.84 N2 -> 7 CO2 + 4 H2O + 33.84 N2
stoich_products = 'CO2:7, H2O:4, N2:33.84'

# Enthalpy accounting:
# h_f(C7H8, liquid) = +12 kJ/mol, gas ≈ +50 kJ/mol
# h_f(CO2) = -393.5 kJ/mol
# h_f(H2O, gas) = -241.8 kJ/mol
# Delta_H_combustion = 7*(-393.5) + 4*(-241.8) - 50 = -3672 kJ/mol toluene

# Heat release per kg of stoich mixture:
MW_mix = (92.14 + 9*32 + 33.84*28) / 1  # g per mol toluene worth of mix
heat_release = 3672e3 / (MW_mix / 1000)  # J/kg of mixture

# Air fuel mass weight ratio
m_air = 9*32 + 33.84*28  # g
m_fuel = 92.14  # g
air_fuel_ratio = m_air / m_fuel

# Set products at room temp, then add combustion enthalpy
gas.TPX = 300, tank_pressure, stoich_products
h_products_cold = gas.enthalpy_mass
h_products_hot = h_products_cold + heat_release

# Equilibrate at constant H, P
gas.HP = h_products_hot, tank_pressure
gas.equilibrate('HP')

T_c = gas.T
gamma = gas.cp / gas.cv
MW = gas.mean_molecular_weight
R_sp = 8314.46 / MW

print("="*50)
print("CHAMBER CONDITIONS (Stoichiometric Toluene-Air)")
print("="*50)
print(f"Chamber pressure:    {tank_pressure/1e5:.1f} bar")
print(f"Chamber temperature: {T_c:.0f} K")
print(f"Gamma:               {gamma:.3f}")
print(f"Mean MW:             {MW:.1f} g/mol")
print()

# Choked flow
mdot = (tank_pressure * A_throat * np.sqrt(gamma / (R_sp * T_c)) 
        * (2/(gamma+1))**((gamma+1)/(2*(gamma-1))))

mdot_air = mdot * (air_fuel_ratio / (1 + air_fuel_ratio))
mdot_fuel = mdot - mdot_air


print(f"Throat area:         {A_throat*1e6:.2f} cm²")
print(f"Mass flow rate:      {mdot*1000:.2f} g/s")
print(f"Air mass flow rate:  {mdot_air*1000:.2f} g/s")
print(f"Fuel mass flow rate: {mdot_fuel*1000:.2f} g/s")
print()

print("Major species (>1%):")
for sp, X in zip(gas.species_names, gas.X):
    if X > 0.01:
        print(f"  {sp:8s}: {X*100:.1f}%")

CHAMBER CONDITIONS (Stoichiometric Toluene-Air)
Chamber pressure:    10.1 bar
Chamber temperature: 2356 K
Gamma:               1.250
Mean MW:             29.4 g/mol

Throat area:         40.00 cm²
Mass flow rate:      32.68 g/s
Air mass flow rate:  30.42 g/s
Fuel mass flow rate: 2.27 g/s

Major species (>1%):
  H2O     : 8.6%
  CO      : 1.1%
  CO2     : 14.4%
  N2      : 74.8%


In [ ]:
import cantera as ct
import numpy as np

# ----- Parameters -----
tank_pressure = 10 * ct.one_atm  # 10 bar
A_throat = 4e-3 * 10e-3  # m²
phi = 0.5  # Equivalence ratio: <1 lean (air-rich), >1 rich (fuel-rich)

# ----- Stoichiometry -----
# C7H8 + 9 O2 -> 7 CO2 + 4 H2O
# With air (21% O2, 79% N2): 9 mol O2 brings 9*(79/21) = 33.857 mol N2
O2_stoich = 9.0
N2_per_O2 = 79.0 / 21.0
N2_stoich = O2_stoich * N2_per_O2

# Molecular weights
MW_toluene = 92.14  # g/mol
MW_O2 = 32.0
MW_N2 = 28.0

# ----- Build reactant mixture -----
# At equivalence ratio phi:
#   phi = (fuel/air)_actual / (fuel/air)_stoich
#   So for fixed 1 mol fuel: air_actual = air_stoich / phi
O2_actual = O2_stoich / phi
N2_actual = O2_actual * N2_per_O2

# Mass calculations
m_fuel = MW_toluene  # g (for 1 mol toluene)
m_air = O2_actual * MW_O2 + N2_actual * MW_N2  # g
m_total = m_fuel + m_air  # g per mol toluene

air_fuel_ratio = m_air / m_fuel
stoich_air_fuel_ratio = (O2_stoich * MW_O2 + N2_stoich * MW_N2) / MW_toluene

# ----- Combustion enthalpy -----
# Heat of combustion for toluene (gas): ~3910 kJ/mol (LHV, products as H2O gas)
# This is released when 1 mol C7H8 reacts with 9 mol O2
delta_H_comb = 3910e3  # J/mol toluene (for complete combustion)

# For rich mixtures, not all fuel burns - scale by available O2
# For lean mixtures, all fuel burns
if phi <= 1.0:
    # Lean/stoich: all fuel burns
    heat_release_per_mol_fuel = delta_H_comb
else:
    # Rich: limited by O2, only fraction (1/phi) of fuel can burn
    heat_release_per_mol_fuel = delta_H_comb / phi

heat_release_per_kg_mix = heat_release_per_mol_fuel / (m_total / 1000)  # J/kg

# ----- Set up Cantera solution -----
gas = ct.Solution('gri30.yaml')

# Build product composition based on stoichiometry
# Lean (phi < 1): excess O2, products are CO2, H2O, O2, N2
# Stoich (phi = 1): products are CO2, H2O, N2
# Rich (phi > 1): excess fuel fragments, products include CO, H2, unburned HC

if phi <= 1.0:
    # Lean or stoichiometric
    # All fuel burns: 7 CO2 + 4 H2O + excess O2 + N2
    n_CO2 = 7.0
    n_H2O = 4.0
    n_O2_excess = O2_actual - O2_stoich  # Excess oxygen
    n_N2 = N2_actual
    
    products = f'CO2:{n_CO2}, H2O:{n_H2O}, N2:{n_N2}'
    if n_O2_excess > 0.01:
        products += f', O2:{n_O2_excess}'
else:
    # Rich mixture - let equilibrium handle the complex chemistry
    # Start with estimated products: partial oxidation + unburned fuel components
    # C7H8 + (9/phi) O2 -> products
    # Available O atoms: 2 * 9 / phi
    O_available = 2 * O2_stoich / phi
    
    # Simplified initial guess (equilibrium will sort it out):
    # Prioritize H2O formation, then CO, with remaining C and H as CO/H2/CH4
    # H in fuel: 8, O needed for H2O: 8 -> 4 H2O uses 4 O
    # Remaining O for carbon: O_available - 8
    
    # This is just an initial state - equilibrium handles the real chemistry
    # Use a mix that Cantera can equilibrate from
    n_H2O = min(4.0, O_available / 2)  # Limited by O or H availability
    O_remaining = O_available - n_H2O
    n_CO = min(7.0, O_remaining)  # CO formation
    O_remaining -= n_CO
    n_CO2 = O_remaining / 2  # Any remaining O makes CO2
    
    n_H2 = 4.0 - n_H2O  # Remaining H as H2
    n_C_remaining = 7.0 - n_CO - n_CO2  # Soot/solid C not in GRI30, use CH4 proxy
    n_N2 = N2_actual
    
    products = f'CO:{max(0,n_CO):.4f}, CO2:{max(0,n_CO2):.4f}, H2O:{max(0,n_H2O):.4f}, H2:{max(0,n_H2):.4f}, N2:{n_N2:.4f}'
    if n_C_remaining > 0.1:
        # Add some CH4 as proxy for unburned hydrocarbons
        products += f', CH4:{n_C_remaining:.4f}'

# Set initial state and add combustion enthalpy
gas.TPX = 300, tank_pressure, products
h_cold = gas.enthalpy_mass
h_hot = h_cold + heat_release_per_kg_mix

# Equilibrate at constant enthalpy and pressure
gas.HP = h_hot, tank_pressure
gas.equilibrate('HP')

T_c = gas.T
gamma = gas.cp / gas.cv
MW = gas.mean_molecular_weight
R_sp = ct.gas_constant / MW  # J/(kg·K)

# ----- Output -----
print("=" * 60)
print(f"CHAMBER CONDITIONS - Toluene/Air Combustion (φ = {phi:.2f})")
print("=" * 60)
if phi < 1:
    print(f"Mixture type:        LEAN (air-rich)")
elif phi > 1:
    print(f"Mixture type:        RICH (fuel-rich)")
else:
    print(f"Mixture type:        STOICHIOMETRIC")
print()
print(f"Equivalence ratio:   {phi:.2f}")
print(f"Air/fuel ratio:      {air_fuel_ratio:.1f} (stoich: {stoich_air_fuel_ratio:.1f})")
print(f"Chamber pressure:    {tank_pressure/1e5:.1f} bar")
print(f"Chamber temperature: {T_c:.0f} K")
print(f"Gamma:               {gamma:.3f}")
print(f"Mean MW:             {MW:.2f} g/mol")
print(f"Specific gas const:  {R_sp:.1f} J/(kg·K)")
print()

# Choked flow calculation
mdot = (tank_pressure * A_throat * np.sqrt(gamma / (R_sp * T_c)) 
        * (2/(gamma+1))**((gamma+1)/(2*(gamma-1))))

mdot_fuel = mdot / (1 + air_fuel_ratio)
mdot_air = mdot - mdot_fuel

print(f"Throat area:         {A_throat*1e6:.1f} mm²")
print(f"Mass flow rate:      {mdot*1000:.2f} g/s")
print(f"Air mass flow rate:  {mdot_air*1000:.2f} g/s")
print(f"Fuel mass flow rate: {mdot_fuel*1000:.2f} g/s")
print()

print("Equilibrium composition (>0.5%):")
for sp, X in sorted(zip(gas.species_names, gas.X), key=lambda x: -x[1]):
    if X > 0.005:
        print(f"  {sp:10s}: {X*100:6.2f}%")

# Additional useful parameters
c_star = tank_pressure * A_throat / mdot  # Characteristic velocity
print()
print(f"Characteristic velocity c*: {c_star:.0f} m/s")

CHAMBER CONDITIONS - Toluene/Air Combustion (φ = 0.90)
Mixture type:        LEAN (air-rich)

Equivalence ratio:   0.90
Air/fuel ratio:      14.9 (stoich: 13.4)
Chamber pressure:    10.1 bar
Chamber temperature: 2341 K
Gamma:               1.253
Mean MW:             29.45 g/mol
Specific gas const:  282.4 J/(kg·K)

Throat area:         40.0 mm²
Mass flow rate:      32.83 g/s
Air mass flow rate:  30.77 g/s
Fuel mass flow rate: 2.06 g/s

Equilibrium composition (>0.5%):
  N2        :  75.31%
  CO2       :  13.60%
  H2O       :   7.87%
  O2        :   1.94%
  NO        :   0.51%

Characteristic velocity c*: 1235 m/s


In [30]:
## Calculate flow rate in terms of volumetric flow at STP
R_universal = 8314.46  # J/kmol-K
T_stp = 273.15  # K
P_stp = 101325.0  # Pa
Vdot_stp = (mdot / gas.mean_molecular_weight) * R_universal * T_stp / P_stp  # m³/s
Vdot_imperial = Vdot_stp * 34.3157  # ft³/s
print(f"Volumetric flow rate at STP: {Vdot_stp:.6f} m³/s")
print(f"Volumetric flow rate at STP: {Vdot_imperial:.3f} ft³/s")

Volumetric flow rate at STP: 0.024902 m³/s
Volumetric flow rate at STP: 0.855 ft³/s
